In this [experiment](Sentence_Emoji_Generator_with_LSTM.ipynb), I built and trained a classifier using an LSTM, which would read a short sentence and determines the best emoji to go with that sentence, based on the starting code, instructions, and utility functions from the [Convolutional Neural Networks](https://coursera.org/learn/convolutional-neural-networks) course (by DeepLearning.AI on Coursera).

Please input sentences in this cell and run the prediction to see the results:
```
x_test = np.array(['I cannot play baseball today', 
                   'My homework is just too hard',
                   'Joey is just hilarious',
                   'I am sorry baby! I will arrive on time!',
                   'Thanks for writing to me',
                   'Love your dishes - best cake ever!',
                   'I miss you. When are you coming back?'])
```

The emojis returned seem to fit the sentences quite well!

![emoji_output_samples.PNG](emoji_output_samples.PNG)

The LSTM architecture is as follows (image from the Coursera assignment):

![emojifier-v2.png](images%2Femojifier-v2.png)

In [3]:
%load_ext autoreload
%autoreload 2

In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# %cd /content/drive/Othercomputers/My Laptop/T_Repo/C5_W2_Assignment2/Files/home/jovyan/work/W2A2
# %ls

/content/drive/Othercomputers/My Laptop/T_Repo/C5_W2_Assignment2/Files/home/jovyan/work/W2A2
data/                 images/                    PythonScript/
Emoji_v3a.ipynb       notebook2script.py         readme.md
emo_utils.py          O_greyhat_Emoji_v3a.ipynb  Sentence_Emoji_Generator_with_LSTM.ipynb
generateTestCases.py  __pycache__/               test_utils.py


In [5]:
%pip install emoji

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 397.5/397.5 kB 5.1 MB/s eta 0:00:00


In [6]:
import numpy as np
from emo_utils import *
import emoji
import matplotlib.pyplot as plt
from test_utils import *

%matplotlib inline

# Dataset
+ X contains 127 sentences
+ Y contains an integer label between 0 and 4 corresponding to an emoji for each sentence

In [7]:
X_train, Y_train = read_csv('data/train_emoji.csv')
X_test, Y_test = read_csv('data/tesss.csv')

In [8]:
maxLen = len(max(X_train, key=len).split())

In [9]:
# Example input strings and associated emojis:
for idx in range(10):
    print(X_train[idx], label_to_emoji(Y_train[idx]))

never talk to me again 😞
I am proud of your achievements 😄
It is the worst day in my life 😞
Miss you so much ❤️
food is life 🍴
I love you mum ❤️
Stop saying bullshit 😞
congratulations on your acceptance 😄
The assignment is too long  😞
I want to go play ⚾


In [10]:
# Convert Y from current shape of (m, 1) into a "one-hot representation" - shape (m, 5)
Y_oh_train = convert_to_one_hot(Y_train, C = 5)
Y_oh_test = convert_to_one_hot(Y_test, C = 5)

In [11]:
# Example of one-hot encoding:
idx = 47
print(f"Sentence '{X_train[idx]}' has label index {Y_train[idx]}, which is emoji {label_to_emoji(Y_train[idx])}", )
print(f"Label index {Y_train[idx]} in one-hot encoding format is {Y_oh_train[idx]}")

Sentence 'this joke is killing me haha' has label index 2, which is emoji 😄
Label index 2 in one-hot encoding format is [0. 0. 1. 0. 0.]


In [12]:
# Loading the word embeddings (GloVe)
word_to_index, index_to_word, word_to_vec_map = read_glove_vecs('data/glove.6B.50d.txt')

- `word_to_index`: dictionary mapping from words to their indices in the vocabulary
    - (400,001 words, with the valid indices ranging from 0 to 400,000)
- `index_to_word`: dictionary mapping from indices to their corresponding words in the vocabulary
- `word_to_vec_map`: dictionary mapping words to their GloVe vector representation.

In [13]:
# Examples
word = "cucumber"
idx = 289846
print("the index of", word, "in the vocabulary is", word_to_index[word])
print("the", str(idx) + "th word in the vocabulary is", index_to_word[idx])

the index of cucumber in the vocabulary is 113317
the 289846th word in the vocabulary is potatos


In [14]:
import numpy as np
import tensorflow
np.random.seed(0)
from keras.api._v2.keras.models import Model
from keras.api._v2.keras.layers import Dense, Input, Dropout, LSTM, Activation
from keras.api._v2.keras.layers import Embedding
from keras.api._v2.keras.preprocessing import sequence
from keras.api._v2.keras.initializers import glorot_uniform
np.random.seed(1)

In [15]:
def sentences_to_indices(X, word_to_index, max_len):
    """
    Converts an array of sentences (strings) into an array of indices corresponding to words in the sentences.
    The output shape should be such that it can be given to `Embedding()` (described in Figure 4).

    Arguments:
    X -- array of sentences (strings), of shape (m, 1)
    word_to_index -- a dictionary containing the each word mapped to its index
    max_len -- maximum number of words in a sentence. You can assume every sentence in X is no longer than this.

    Returns:
    X_indices -- array of indices corresponding to words in the sentences from X, of shape (m, max_len)
    """

    m = X.shape[0]

    X_indices = np.zeros([m,max_len])

    for i in range(m):                               # loop over training examples

        # Convert the ith training sentence in lower case and split is into words. You should get a list of words.
        sentence_words = X[i].lower().split()

        j = 0

        for w in sentence_words:
            if w in word_to_index:
                X_indices[i, j] = word_to_index[w]
                j += 1
    return X_indices

In [16]:
# Examples
X1 = np.array(["funny lol", "lets play baseball", "food is ready for you"])
X1_indices = sentences_to_indices(X1, word_to_index, max_len=5)
print("X1 =", X1)
print("X1_indices =\n", X1_indices)

X1 = ['funny lol' 'lets play baseball' 'food is ready for you']
X1_indices =
 [[155345. 225122.      0.      0.      0.]
 [220930. 286375.  69714.      0.      0.]
 [151204. 192973. 302254. 151349. 394475.]]


# Creating Keras embedding layer with pre-trained weights

In [17]:
def pretrained_embedding_layer(word_to_vec_map, word_to_index):
    """
    Creates a Keras Embedding() layer and loads in pre-trained GloVe 50-dimensional vectors.

    Arguments:
    word_to_vec_map -- dictionary mapping words to their GloVe vector representation.
    word_to_index -- dictionary mapping from words to their indices in the vocabulary (400,001 words)

    Returns:
    embedding_layer -- pretrained layer Keras instance
    """

    vocab_size = len(word_to_index) + 1              # adding 1 to fit Keras embedding (requirement)
    any_word = list(word_to_vec_map.keys())[0]
    emb_dim = word_to_vec_map[any_word].shape[0]    # define dimensionality of your GloVe word vectors (= 50)

    # Step 1
    # Initialize the embedding matrix as a numpy array of zeros.
    # See instructions above to choose the correct shape.
    emb_matrix = np.zeros([vocab_size,emb_dim])

    # Step 2
    # Set each row "idx" of the embedding matrix to be
    # the word vector representation of the idx'th word of the vocabulary
    for word, idx in word_to_index.items():
        emb_matrix[idx, :] = word_to_vec_map[word]

    # Step 3
    # Define Keras embedding layer with the correct input and output sizes
    # Make it non-trainable.
    embedding_layer = Embedding(vocab_size, emb_dim ,trainable = False)

    # Build the embedding layer, it is required before setting the weights of the embedding layer.
    embedding_layer.build((None,))

    # Set the weights of the embedding layer to the embedding matrix. Your layer is now pretrained.
    embedding_layer.set_weights([emb_matrix])

    return embedding_layer

# Building the LSTM neural network

![emojifier-v2.png](images%2Femojifier-v2.png)

In [18]:
def Emojify(input_shape, word_to_vec_map, word_to_index):
    """
    Function creating the Emojify-v2 model's graph.

    Arguments:
    input_shape -- shape of the input, usually (max_len,)
    word_to_vec_map -- dictionary mapping every word in a vocabulary into its 50-dimensional vector representation
    word_to_index -- dictionary mapping from words to their indices in the vocabulary (400,001 words)

    Returns:
    model -- a model instance in Keras
    """

    # Define sentence_indices as the input of the graph.
    # It should be of shape input_shape and dtype 'int32' (as it contains indices, which are integers).
    sentence_indices = Input(shape=input_shape,dtype='int32')

    # Create the embedding layer pretrained with GloVe Vectors (≈1 line)
    embedding_layer = pretrained_embedding_layer(word_to_vec_map, word_to_index)

    # Propagate sentence_indices through your embedding layer
    # (See additional hints in the instructions).
    embeddings = embedding_layer(sentence_indices)

    # Propagate the embeddings through an LSTM layer with 128-dimensional hidden state
    # The returned output should be a batch of sequences, So, set return_sequences = True
    # If return_sequences = False, the LSTM returns only tht last output in output sequence
    X = LSTM(units=128, return_sequences=True)(embeddings)
    # Add dropout with a probability of 0.5
    X = Dropout(0.5)(X)
    # Propagate X trough another LSTM layer with 128-dimensional hidden state
    # The returned output should be a single hidden state, not a batch of sequences.
    X = LSTM(units=128, return_sequences=False)(X)
    # Add dropout with a probability of 0.5
    X = Dropout(0.5)(X)
    # Propagate X through a Dense layer with 5 units
    X = Dense(5)(X)
    # Add a softmax activation
    X = Activation('softmax')(X)

    # Create Model instance which converts sentence_indices into X.
    return Model(inputs=sentence_indices,outputs=X)


In [19]:
model = Emojify((maxLen,), word_to_vec_map, word_to_index)
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 10)]              0         
                                                                 
 embedding (Embedding)       (None, 10, 50)            20000050  
                                                                 
 lstm (LSTM)                 (None, 10, 128)           91648     
                                                                 
 dropout (Dropout)           (None, 10, 128)           0         
                                                                 
 lstm_1 (LSTM)               (None, 128)               131584    
                                                                 
 dropout_1 (Dropout)         (None, 128)               0         
                                                                 
 dense (Dense)               (None, 5)                 645   

In [20]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Training the Model

In [21]:
X_train_indices = sentences_to_indices(X_train, word_to_index, maxLen)
Y_train_oh = convert_to_one_hot(Y_train, C=5)

In [22]:
model.fit(X_train_indices, Y_train_oh, epochs=50, batch_size=32, shuffle=True)

Epoch 1/50
5/5 [==============================] - 5s 16ms/step - loss: 1.5993 - accuracy: 0.2424
Epoch 2/50
5/5 [==============================] - 0s 6ms/step - loss: 1.5107 - accuracy: 0.3636
Epoch 3/50
5/5 [==============================] - 0s 6ms/step - loss: 1.4668 - accuracy: 0.3258
Epoch 4/50
5/5 [==============================] - 0s 6ms/step - loss: 1.4156 - accuracy: 0.3712
Epoch 5/50
5/5 [==============================] - 0s 6ms/step - loss: 1.3028 - accuracy: 0.4924
Epoch 6/50
5/5 [==============================] - 0s 6ms/step - loss: 1.1654 - accuracy: 0.5530
Epoch 7/50
5/5 [==============================] - 0s 6ms/step - loss: 1.0482 - accuracy: 0.6061
Epoch 8/50
5/5 [==============================] - 0s 6ms/step - loss: 0.9967 - accuracy: 0.6061
Epoch 9/50
5/5 [==============================] - 0s 6ms/step - loss: 0.8325 - accuracy: 0.6894
Epoch 10/50
5/5 [==============================] - 0s 6ms/step - loss: 0.7707 - accuracy: 0.7500
Epoch 11/50
5/5 [=====================

In [23]:
# Testing accuracy
X_test_indices = sentences_to_indices(X_test, word_to_index, max_len = maxLen)
Y_test_oh = convert_to_one_hot(Y_test, C = 5)
loss, acc = model.evaluate(X_test_indices, Y_test_oh)
print()
print("Test accuracy = ", acc)

2/2 [==============================] - 1s 7ms/step - loss: 0.6295 - accuracy: 0.8036

Test accuracy =  0.8035714030265808


In [33]:
# Now trying with new sentences
x_test = np.array(['I cannot play baseball today',
                   'My homework is just too hard',
                   'Joey is just hilarious',
                   'I am sorry baby! I will arrive on time!',
                   'Thanks for writing to me',
                   'Love your dishes - best cake ever!',
                   'I miss you. When are you coming back?'])
X_test_indices = sentences_to_indices(x_test, word_to_index, maxLen)
# print(X_test_indices.shape)
preds = model.predict(X_test_indices)
for i, pred in enumerate(preds):
  print(x_test[i] +' '+  label_to_emoji(np.argmax(pred)))

1/1 [==============================] - 0s 19ms/step
I cannot play baseball today ⚾
My homework is just too hard 😞
Joey is just hilarious 😄
I am sorry baby! I will arrive on time! 😞
Thanks for writing to me 😄
Love your dishes - best cake ever! 🍴
I miss you. When are you coming back? ❤️
